In [58]:
#!pip install yfinance


In [59]:
#!pip install pyportfolioopt


In [399]:
import yfinance as yf
import pandas as pd
import numpy as np
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.hierarchical_portfolio import HRPOpt
from pypfopt.cla import CLA
from pypfopt import risk_models
from pypfopt import expected_returns
import datetime as dt
from dateutil.relativedelta import relativedelta

In [234]:
tickers = ['SPY', 'EEM', 'TIP', 'IAU', 'DBC', 'TLT', 'VGK', 'IWM','LQD','VNQ', 'SHY', 'IWM','QQQ','HYG']

In [235]:
prices = yf.download(tickers=tickers)

[*********************100%%**********************]  13 of 13 completed


In [236]:
prices.to_csv("price_data.csv")

In [429]:
subset_tickers = ['SPY', 'EEM', 'TIP', 'IAU', 'DBC', 'TLT', 'VGK', 'IWM','LQD','VNQ', 'SHY','QQQ','HYG']

In [430]:
# Preprocess df for adjusted close only
adjclose = prices[prices.columns[:len(tickers)+1]]
adjclose.columns = [x[1] for x in adjclose.columns]
adjclose = adjclose.reset_index().rename(columns={'Date':'date'})
adjclose['date'] = pd.to_datetime(adjclose['date']).dt.date
adjclose.set_index(['date'], inplace=True)
adjclose = adjclose[subset_tickers]
# Check the start dates
for c in adjclose.columns:
    print(c , ": ", adjclose[c].first_valid_index())
adjclose.dropna(inplace=True)

SPY :  1993-01-29
EEM :  2003-04-14
TIP :  2003-12-05
IAU :  2005-01-28
DBC :  2006-02-06
DBC :  2006-02-06
TLT :  2002-07-30
VGK :  2005-03-10
IWM :  2000-05-26
LQD :  2002-07-30
VNQ :  2004-09-29
SHY :  2002-07-30
QQQ :  1999-03-10
HYG :  2007-04-11


In [424]:
# We create a rebalance logic for historical backtest style analyis
rebal_date = dt.datetime.strptime('2012-01-01', '%Y-%m-%d').date()
port_wts = {}
for ddate in adjclose.index.unique():
    if ddate>= rebal_date:
        adjclose_t = adjclose.loc[(adjclose.index <= ddate)]
        mu = expected_returns.mean_historical_return(adjclose_t)
        S = risk_models.exp_cov(adjclose_t, span = 180)
       # ret = adjclose_t.shift(-1)/adjclose_t-1
        ef = EfficientFrontier(mu, S,weight_bounds=(0,0.25))
        weights = ef.efficient_risk(target_volatility=0.1)   
        port_wts[ddate] = weights
        rebal_date = rebal_date + relativedelta(months=1)
    

In [431]:
# Portfolio processing
portfolio_wts = pd.DataFrame(port_wts).T
adjclose_return = adjclose.loc[(adjclose.index.isin(portfolio_wts.index))].shift(-1)/adjclose.loc[(adjclose.index.isin(portfolio_wts.index))]-1
fwd_return = adjclose.loc[(adjclose.index.isin(portfolio_wts.index))].shift(-1)/adjclose.loc[(adjclose.index.isin(portfolio_wts.index))]
portfolio_wts[portfolio_wts<0.01] = 0
port_ret = portfolio_wts*adjclose_return
cumulative_return_series = (port_ret.sum(axis=1)+1).cumprod()
cumulative_return = (cumulative_return.tail(1)-1)
annualized_ret = (cumulative_return_series.tail(1).values[0]**(12/len(cumulative_return_series))-1)
annualized_vol =  port_ret.sum(axis=1).std()*np.sqrt(12)

In [427]:
print("Cumulative return: " , cumulative_return)
print("Portfolio Annual returs: ",annualized_ret )
print("Portfolio Vol: ",annualized_vol)
print("Portfolio sharpe: ", annualized_ret/annualized_vol)

Cumulative return:  2023-11-01   -0.461971
dtype: float64
Portfolio Annual returs:  0.04348516838239025
Portfolio Vol:  0.0913396068122931
Portfolio sharpe:  0.476082281279732
